In [2]:
import pandas as pd
import numpy as np
from numpy import log
from matplotlib import pyplot as plt 
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import normaltest

import warnings
warnings.filterwarnings("ignore")

In [11]:
data = pd.read_csv(r'data\owid-covid-data.csv')
# 計算 iso_code 的數量，排除以 "OWID" 開頭的代碼
filtered_iso_codes = data["iso_code"].value_counts()
filtered_iso_codes = filtered_iso_codes[~filtered_iso_codes.index.str.startswith("OWID")]

# 取出前 10 至 20 的地區代號並轉換為列表
iso_code_list = filtered_iso_codes.iloc[:30].index.tolist()

# 顯示結果
print(iso_code_list)

['ARG', 'MEX', 'THA', 'TWN', 'CHN', 'MAC', 'KOR', 'JPN', 'USA', 'CAN', 'VNM', 'SGP', 'HKG', 'MYS', 'FRA', 'NPL', 'AUS', 'LKA', 'DEU', 'KHM', 'ARE', 'FIN', 'IND', 'PHL', 'FJI', 'GBR', 'RUS', 'ITA', 'SWE', 'ESP']


In [16]:
from sklearn.preprocessing import StandardScaler

def standardize_data(data: pd.DataFrame, column: str):
    """
    使用 z-score 進行標準化處理
    """
    scaler = StandardScaler()
    data["standardized_" + column] = scaler.fit_transform(data[column].values.reshape(-1, 1))
    return data, scaler

def log_transform(data: pd.DataFrame, column: str):
    data["log_" + column] = log(data[column] + 0.01)
    return data

def adf_test(data: pd.DataFrame, column: str):
    data.dropna(inplace=True)
    result = adfuller(data[column])
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    if result[1] < 0.05:
        print("數據是穩態的")
    else:
        print("數據是非穩態的，需要進行差分")

    # 3. 繪製 ACF 和 PACF 檢查自相關性
    # plt.figure(figsize=(8, 4))
    # plot_acf(data["new_cases"], lags=30, ax=plt.subplot(121))
    # plot_pacf(data["new_cases"], lags=30, ax=plt.subplot(122))
    # plt.tight_layout()
    # plt.show()
def calculate_mape(true_values, predictions):
    """
    計算 Mean Absolute Percentage Error (MAPE)
    """
    true_values = np.array(true_values)
    predictions = np.array(predictions)
    return np.mean(np.abs((true_values - predictions) / true_values)) * 100

def calculate_ppe(true_values, predictions):
    """
    計算 Peak Prediction Error (PPE)
    包括高峰值大小和高峰時間點的誤差
    """
    peak_true_value = np.max(true_values)
    peak_true_time = np.argmax(true_values)

    peak_pred_value = np.max(predictions)
    peak_pred_time = np.argmax(predictions)

    # 高峰值誤差
    value_error = np.abs(peak_true_value - peak_pred_value)

    # 高峰時間誤差
    time_error = np.abs(peak_true_time - peak_pred_time)

    return value_error, time_error

def grid_search_ARIMA(data: pd.DataFrame, column: str):
    best_model = None
    best_order = None
    best_aic = float("inf")

    for p in range(3):
        for d in range(3):
            for q in range(3):
                try:
                    model = ARIMA(data[column], order=(p, d, q))
                    model_fit = model.fit()

                    if model_fit.aic < best_aic:
                        best_model = model_fit
                        best_order = (p, d, q)
                        best_aic = model_fit.aic
                except Exception as e:
                    # 捕捉錯誤，但繼續進行其他組合的測試
                    print(f"Error for order ({p}, {d}, {q}): {e}")
                    continue

    # if best_model is not None:
        # print("Best Model Summary:")
        # print(best_model.summary())

    return best_order

def rolling_forecast(data: pd.DataFrame, column: str, best_order: tuple):
    # 劃分數據集
    train_size = int(len(data) * 0.8 // 1)  # 使用前 80% 的數據作為訓練集
    train = data.iloc[:train_size]     # 訓練集
    test = data.iloc[train_size:]      # 測試集

    print(f"Train Size: {len(train)}, Test Size: {len(test)}")

    # 初始化滾動預測
    history = [x for x in train[column]]  # 使用訓練集的初始數據
    predictions = []  # 儲存每次的預測結果

    # 滾動預測
    for t in range(len(test)):
        # 構建並擬合 ARIMA 模型
        model = ARIMA(history, order=best_order)
        model_fit = model.fit()

        # 預測下一個時間點
        yhat = model_fit.forecast(steps=1)[0]
        predictions.append(yhat)

        # 將真實值加入訓練集
        history.append(test[column].iloc[t])

    # 計算各種指標
    mae = mean_absolute_error(test[column], predictions)
    r2 = r2_score(test[column], predictions)
    rmse = np.sqrt(mean_squared_error(test[column], predictions))

    print(f"Mean of Real Data: {np.mean(test[column])}")
    print(f"Median of Real Data: {np.median(test[column])}")
    print(f"Mean Absolute Error: {mae}") # 平均絕對誤差
    print(f"Percentage of MAE: {mae / np.mean(test[column])}") # 平均絕對誤差占比
    print(f"Rolling Forecast RMSE: {rmse}") # RMSE

def check_residuals(data: pd.DataFrame, column: str, best_order: tuple):
    # 檢查殘差的自相關性
    model = ARIMA(data[column], order=best_order)
    model_fit = model.fit()
    residuals = model_fit.resid
    lb_test = acorr_ljungbox(residuals, lags=[10], return_df=True)
    print(lb_test)

    # 檢查殘差的正態分佈
    stat, p = normaltest(residuals)
    print(f"Normality Test: Statistic={stat}, p={p}")
    if p > 0.05:
        print("殘差接近正態分佈")
    else:
        print("殘差偏離正態分佈")

def rolling_window_forecast(data: pd.DataFrame, column: str, best_order: tuple, input_weeks: int = 4, forecast_weeks: int = 1, scaler=None):
    # 計算窗口大小
    input_days = input_weeks * 7
    forecast_days = forecast_weeks * 7

    # 初始化預測結果和真實值
    predictions = []
    true_values = []

    # 滾動窗口預測
    for start in range(len(data) - input_days - forecast_days + 1):
        # 定義訓練窗口和測試窗口
        train_window = data[column].iloc[start:start + input_days]
        test_window = data[column].iloc[start + input_days:start + input_days + forecast_days]

        # 構建並擬合 ARIMA 模型
        model = ARIMA(train_window, order=best_order)
        model_fit = model.fit()

        # 預測未來數據
        forecast = model_fit.forecast(steps=forecast_days)
        predictions.extend(forecast)
        true_values.extend(test_window)

    # 將標準化數據還原到原始尺度
    if scaler:
        predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
        true_values = scaler.inverse_transform(np.array(true_values).reshape(-1, 1)).flatten()

    # 計算性能指標
    rmse = np.sqrt(mean_squared_error(true_values, predictions))
    mae = mean_absolute_error(true_values, predictions)
    mape = calculate_mape(true_values, predictions)
    ppe_value_error, ppe_time_error = calculate_ppe(true_values, predictions)

    # 輸出結果
    print(f"MAE: {mae}")
    print(f"MAE / Mean: {mae / np.mean(true_values)}")
    print(f"RMSE: {rmse}")
    print(f"MAPE: {mape:.2f}%")
    print(f"Peak Prediction Error (Value): {ppe_value_error}")
    print(f"Peak Prediction Error (Time): {ppe_time_error} days")

def pipeline(data: pd.DataFrame, column: str, forecast_steps: int):
    # 1. 標準化數據
    data, scaler = standardize_data(data, column)

    # 2. ADF 單根檢定
    adf_test(data, "standardized_" + column)

    # 3. 使用網格搜索尋找最佳參數
    print("----------------Grid Search ARIMA------------------")
    best_order = grid_search_ARIMA(data, "standardized_" + column)

    # 4. 滾動窗口多步預測
    print("----------------Multi-step Forecast------------------")
    rolling_window_forecast(data, "standardized_" + column, best_order, input_weeks=forecast_steps, scaler=scaler)

In [18]:
regions = ["MYS","MEX", "THA", "TWN", "CHN","JPN", "CAN", "VNM", "SGP", "ARG"]# 
# Exclude the region in iso_code_list that is not in the regions list
new_regions = [region for region in iso_code_list if region not in regions] 

In [19]:
for region in new_regions:
    print(f"--------------------------------------Region: {region}--------------------------------------")
    data = pd.read_csv(r"data\owid-covid-data.csv")
    data = data[data["iso_code"] == region]
    data = data[["date", "new_cases_smoothed"]]
    pipeline(data, "new_cases_smoothed", 4)

--------------------------------------Region: MAC--------------------------------------
ADF Statistic: -5.271333
p-value: 0.000006
Critical Values:
	1%: -3.439
	5%: -2.865
	10%: -2.569
數據是穩態的
----------------Grid Search ARIMA------------------
----------------Multi-step Forecast------------------
MAE: 0.07278307215684197
MAE / Mean: 0.7538732137002395
RMSE: 0.20630347725856468
MAPE: nan%
Peak Prediction Error (Value): 0.9914964651581526
Peak Prediction Error (Time): 14 days
--------------------------------------Region: KOR--------------------------------------
ADF Statistic: -0.076716
p-value: 0.951709
Critical Values:
	1%: -3.439
	5%: -2.865
	10%: -2.569
數據是非穩態的，需要進行差分
----------------Grid Search ARIMA------------------
----------------Multi-step Forecast------------------
MAE: 347.4653072960881
MAE / Mean: 0.08020363554663068
RMSE: 1896.6099199670773
MAPE: 14.46%
Peak Prediction Error (Value): 1891.9656557179405
Peak Prediction Error (Time): 0 days
-----------------------------------